In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library
from pandas import read_html

print('Libraries imported.')

Libraries imported.


### Using Pandas.read_html() Function to extract the wikitable from the website
 I found this approach to be easier to implement compared to using BeautifulSoup

In [2]:
page='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wikitable = read_html(page, index_col=0, attrs={"class":"wikitable sortable"})
print("Extracted {num} wikitables".format(num=len(wikitable)))

Extracted 1 wikitables


In [3]:
df=wikitable[0]

In [4]:
df.head()

,1,2
0,,
Postcode,Borough,Neighbourhood
M1A,Not assigned,Not assigned
M2A,Not assigned,Not assigned
M3A,North York,Parkwoods
M4A,North York,Victoria Village


In [5]:
df=df.drop(['Postcode'],axis=0)

In [6]:
df=df.reset_index()

Renaming the columns of the dataset

In [7]:
df=df.rename(index=str, columns={0: "Postcode", 1: "Borough",2:"Neighborhood"})

In [8]:
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [9]:
df.shape

(289, 3)

#### Dropping the rows that have Borough values as Not Assigned and copying the Borough values to Neighborhood if Neighborhood is Not Assigned

In [10]:
for x in range(0,289) :
    i=str(x)
    if df.loc[i,'Borough'] == "Not assigned" :
        df=df.drop([i],axis=0)
    elif df.loc[i,'Neighborhood'] == "Not assigned" :
        df.loc[i,'Neighborhood'] = df.loc[i,'Borough']

In [11]:
df.head()

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [12]:
df.shape

(212, 3)

In [13]:
unique = [] 
for x in df['Postcode']: 
    if x not in unique: 
        unique.append(x) 
print(len(unique))

103


We have 103 unique postcodes

In [14]:
df2=df

#### A Loop to merge neighborhoods 

In [15]:
postcode = unique
borough=[]
neighborhood=[]
ind=0
for x in postcode :
    length=len(df2[df2['Postcode'] == x].index)
    indexs=df2[df2['Postcode'] == x].index
    if length == 1 :
        borough.append(df.loc[str(indexs[0]),'Borough'])
        neighborhood.append(df.loc[str(indexs[0]),'Neighborhood'])
    else :
        borough.append(df.loc[str(indexs[0]),'Borough'])
        neighborhood.append(df2.loc[str(indexs[0]),'Neighborhood'])
        for i in indexs[1:]:
            pos=str(i)
            neighborhood[ind]=neighborhood[ind]+', '+df2.loc[pos,'Neighborhood']
    ind=ind+1

In [16]:
print(len(postcode),len(borough),len(neighborhood))

103 103 103


In [17]:
df2=pd.DataFrame({'Postcode':postcode,'Borough':borough,"Neighborhood":neighborhood})

DF2 is the Dataframe containing the merged neighborhoods

In [18]:
df2.head()

,Borough,Neighborhood,Postcode
0,North York,Parkwoods,M3A
1,North York,Victoria Village,M4A
2,Downtown Toronto,"Harbourfront, Regent Park",M5A
3,North York,"Lawrence Heights, Lawrence Manor",M6A
4,Queen's Park,Queen's Park,M7A


In [19]:
df2.shape

(103, 3)

I have dropped all the not assigned rows and hence Queen's Par